
# 本番運用の品質モニタリング (スコアラーの自動実行)

https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/run-scorer-in-prod

In [0]:
%pip install -U "mlflow[databricks]>=3.1.1" openai

%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Traceデータの作成

In [0]:
import mlflow
from openai import OpenAI
from mlflow.entities import Document
from typing import List, Dict

# OpenAI呼び出しの自動トレースを有効にする
mlflow.openai.autolog()

# 同じ資格情報を使用してOpenAI経由でDatabricks LLMに接続する
# あるいは、ここで独自のOpenAI資格情報を使用することもできます
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
)

@mlflow.trace
def generate_feature_explanation(system_name:str, feature_name: str) -> Dict[str, str]:
    """顧客データと営業担当者の指示に基づいてパーソナライズされた営業メールを生成する"""

    # 取得したコンテキストを使用してメールを生成する
    prompt = f"""あなたは機械学習やデータエンジニアリングのエキスパートです。
    下記のシステム名および機能名で示される機能について、解説してください。

    システム名:
    {system_name}

    機能名:
    {feature_name}
    
    説明は簡潔かつ箇条書きでまとめてください。"""

    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {"role": "system", "content": "あなたは役に立つアシスタントです。"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )

    return {"explanation": response.choices[0].message.content}


# トレースを作成する
generate_feature_explanation("MLflow", "ChatAgent")
generate_feature_explanation("MLflow", "Tracing")
generate_feature_explanation("Databricks", "Unity Catalog")
generate_feature_explanation("Databricks", "Agent Evaluation")
generate_feature_explanation("PowerBI", "Dashboard")


{'explanation': 'PowerBIのダッシュボード機能について解説します。\n\nダッシュボードは、PowerBIの重要な機能の一つです。以下はその主な特徴と利点です。\n\n*   クイックビューのための設計: ダッシュボードは、重要な指標やデータを一目で把握できるように設計されています。\n*   カスタマイズ可能: ユーザーは、ダッシュボードに表示されるデータやビジュアルを自由に選び、自分のニーズに合わせてカスタマイズできます。\n*   インタラクティブ: ダッシュボードはインタラクティブです。ユーザーは、ビジュアルの詳細を表示したり、別のビューに切り替えたりできるため、データの分析が深みを増します。\n*   共有とコラボレーション: ダッシュボードを他のユーザーと共有することができます。さらに、リアルタイムで更新されるため、チームメンバー間でのコラボレーションや意思決定を促進します。\n*   モバイルアクセス: PowerBIのダッシュボードはモバイルデバイスからもアクセス可能です。これにより、ユーザーはどこからでも重要なデータにアクセスできます。'}

[Trace(trace_id=tr-09a1f6a4ae44a57e8c054af1bfb61939), Trace(trace_id=tr-5a2e311b290434d46d9b66ee0188c293), Trace(trace_id=tr-0f4e21012373f9c0e23f8f08fcfd34ea), Trace(trace_id=tr-e21e818c9bf8d949c27b04414b7ea278), Trace(trace_id=tr-dfcce47f4a6fd59ce5e309f2d8f73df5)]

In [0]:
import mlflow

from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
)

# 実験から最大10件のトレースをサンプリング
traces = mlflow.search_traces(max_results=10)

# スコアラーを使って評価を実行
with mlflow.start_run(run_name="eval_v1"):
    mlflow.genai.evaluate(
        data=traces,
        scorers=[
            RelevanceToQuery(),
            Guidelines(
                name="mlflow_only",
                # ガイドラインはリクエストとレスポンスを参照可能
                guidelines="リクエストがMLflowに関連していない場合、応答は回答を拒否しなければなりません。",
            ),
            # ガイドラインは複数指定可能
            Guidelines(
                name="customer_service_tone",
                guidelines="""応答は次のブランドボイスを維持しなければなりません:
        - プロフェッショナルでありながら温かく会話的（企業用語を避ける）
        - 感情的な文脈を認識し、解決策に飛びつく前に共感を示す
        - 押し付けがましくなく積極的に助けを提供する

        具体的には:
        - 顧客がフラストレーション、怒り、失望を表明した場合、最初の文は彼らの感情を認識しなければなりません
        - 応答は「私」声明を使用して所有権を取る必要があります（例：「私は理解します」ではなく「私たちは理解します」）
        - 応答は「単に」、「ただ」、「明らかに」などの懸念を最小限に抑えるフレーズを避けなければなりません
        - 応答は具体的な次のステップまたは助けを提供するオープンエンドの提案で終わらなければなりません、一般的な締めくくりではありません""",
            ),
        ],
    )

Evaluating:   0%|          | 0/5 [Elapsed: 00:00, Remaining: ?] 

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results.

[Trace(trace_id=tr-dfcce47f4a6fd59ce5e309f2d8f73df5), Trace(trace_id=tr-5a2e311b290434d46d9b66ee0188c293), Trace(trace_id=tr-e21e818c9bf8d949c27b04414b7ea278), Trace(trace_id=tr-09a1f6a4ae44a57e8c054af1bfb61939), Trace(trace_id=tr-0f4e21012373f9c0e23f8f08fcfd34ea)]

## ステップ 2: モニタリングを有効にする

それでは、モニタリングサービスを有効にしましょう。 有効にすると、モニタリング サービスは、評価されたトレースのコピーをMLflow エクスペリメントから、指定した スキーマのDelta Unity Catalogテーブルに同期します。

In [0]:
# これらのパッケージは mlflow[databricks] で自動的にインストールされます
from databricks.agents.monitoring import (
    create_external_monitor,
    AssessmentsSuiteConfig,
    BuiltinJudge,
    GuidelinesJudge,
)

external_monitor = create_external_monitor(
    # CREATE TABLE 権限のある Unity Catalog スキーマに変更してください
    catalog_name="workspace",
    schema_name="default",
    assessments_config=AssessmentsSuiteConfig(
        sample=1.0,  # サンプリングレート
        assessments=[
            BuiltinJudge(
                name="relevance_to_query"
            ),  # または {'name': 'relevance_to_query'}
            # ガイドラインはリクエストとレスポンスを参照可能
            GuidelinesJudge(
                guidelines={
                    # 任意の数のガイドラインを key-value で指定可能
                    "mlflow_only": [
                        "リクエストがMLflowに関連していない場合、応答は回答を拒否しなければなりません。"
                    ],  # 配列で指定
                    "customer_service_tone": [
                        """応答は次のブランドボイスを維持しなければなりません:
    - プロフェッショナルでありながら温かく会話的（企業用語を避ける）
    - 感情的な文脈を認識し、解決策に飛びつく前に共感を示す
    - 押し付けがましくなく積極的に助けを提供する

    具体的には:
    - 顧客がフラストレーション、怒り、失望を表明した場合、最初の文は彼らの感情を認識しなければなりません
    - 応答は「私」声明を使用して所有権を取る必要があります（例：「私は理解します」ではなく「私たちは理解します」）
    - 応答は「単に」、「ただ」、「明らかに」などの懸念を最小限に抑えるフレーズを避けなければなりません
    - 応答は具体的な次のステップまたは助けを提供するオープンエンドの提案で終わらなければなりません、一般的な締めくくりではありません"""
                    ],
                }
            ),
        ],
    ),
)

print(external_monitor)

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File <command-5874206880854678>, line 9
      1 # これらのパッケージは mlflow[databricks] で自動的にインストールされます
      2 from databricks.agents.monitoring import (
      3     create_external_monitor,
      4     AssessmentsSuiteConfig,
      5     BuiltinJudge,
      6     GuidelinesJudge,
      7 )
----> 9 external_monitor = create_external_monitor(
     10     # CREATE TABLE 権限のある Unity Catalog スキーマに変更してください
     11     catalog_name="workspace",
     12     schema_name="default",
     13     assessments_config=AssessmentsSuiteConfig(
     14         sample=1.0,  # サンプリングレート
     15         assessments=[
     16             BuiltinJudge(
     17                 name="relevance_to_query"
     18             ),  # または {'name': 'relevance_to_query'}
     19             # ガイドラインはリクエストとレスポンスを参照可能
     20             GuidelinesJudge(
     21                


# ステップ4.モニタリング結果の使用
モニタリング ジョブを初めて実行するには、~15 分から 30 分かかります。 最初の実行後、15 分ごとに実行されます。 本番運用のトラフィックが大量にある場合、ジョブの完了にさらに時間がかかる可能性があることに注意してください。

ジョブが実行されるたびに、次のことが行われます。

トレースのサンプルに対して各スコアラーを実行します

スコアラーごとにサンプリングレートが異なる場合、モニタリングジョブは、同じトレースをできるだけ多くスコアリングしようとします。たとえば、スコアラー A のサンプリング レートが 20%で、スコアラー B のサンプリング レートが 40% の場合、トレースの同じ 20% が A と B に使用されます。
スコアラーからの フィードバック を MLflow エクスペリメントの各トレースに添付します

すべてのトレース (サンプリングされたトレースだけでなく) のコピーを、手順 1 で構成された Delta テーブルに書き込みます。

モニタリング結果は、 MLflow エクスペリメントのTraceタブを使用して表示できます。または、生成された Delta テーブルで SQL または Spark を使用してトレースのクエリを実行することもできます。

In [0]:
# 追加のトレースを作成する
generate_feature_explanation("MLflow", "PromptRegistry")
generate_feature_explanation("python", "Pandas")
generate_feature_explanation("python", "numpy")
generate_feature_explanation("Databricks", "Model Serving")
generate_feature_explanation("MLflow", "LLM Evaluation")


{'explanation': 'MLflowにおけるLLM Evaluationの機能について、以下の点が解説できます。\n\n*   LLMとは、大規模言語モデル（Large Language Model）の略称です。\n*   LLM Evaluationは、機械学習モデル、特にLLMの性能を評価するための機能です。\n*   この機能を利用することで、開発者はモデルをトレーニングした後、その性能を客観的に評価できます。\n*   評価には、精度、再現率、F1スコアなどのメトリックが用いられます。\n*   これらのメトリックは、モデルがデータを正しく予測・分類する能力を示します。\n*   LLM Evaluationは、モデル開発の重要なステップであり、開発者がモデルを改善し、最適なパフォーマンスを実現するのに役立ちます。'}

[Trace(trace_id=tr-659451e46cef6b93d6bfd6c60a3cab42), Trace(trace_id=tr-77df998c39304b8020f4684a636ae13a), Trace(trace_id=tr-d73d33f59da306032b787189eacd805c), Trace(trace_id=tr-4974ace0af12050ebbf697a4c717c67e), Trace(trace_id=tr-028b0705090540bae8d21966687c65d2)]